In [2]:
# Standard libraries
import sys
# Add your custom path
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
import os
import logging
import argparse # Argument parsing

# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports

# irr  ar(1) offset logic 이용했을때 사실 차이 없고 latitude centering 했을 때 소수점 10쨰자리에서 살짝 개선


from GEMS_TCO import kernels_gpu_aroffset_latcentering_020326 as kernels_ar  

from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import alg_optimization, BaseLogger
from GEMS_TCO import kernels_columns as kernels_reparam_space_time_gpu_col
from typing import Optional, List, Tuple
from pathlib import Path
import typer
import json
from json import JSONEncoder
from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader_ar import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle
from torch.nn import Parameter

Load daily data applying max-min ordering

In [3]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

lat_range_input=[-3,2]      
lon_range_input=[121, 131] 
lat_range_input=[-3,-1]      
lon_range_input=[121, 125] 

df_map, ord_mm, nns_map, day_offsets = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,

lat_range=lat_range_input,   
lon_range=lon_range_input
  
)


--- [AR(1) Analysis for Mean Function] ---
  Global Monthly Mean (a_y): 257.0667
  Fitted AR(1) Phi: 0.6765


In [4]:
daily_aggregated_tensors_dw = [] 
daily_hourly_maps_dw = []      

daily_aggregated_tensors_vecc = [] 
daily_hourly_maps_vecc = []   

for day_index in range(31):
    hour_start_index = day_index * 8
    hour_end_index = (day_index + 1) * 8
    hour_indices = [hour_start_index, hour_end_index]

    # --- DW용 데이터 로드 (day_offsets 인자 추가) ---
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        day_offsets,  # <--- 이 부분이 추가되어야 합니다
        hour_indices, 
        ord_mm=None,
        dtype=torch.float64, 
        keep_ori=False
    )
    daily_aggregated_tensors_dw.append(day_aggregated_tensor)
    daily_hourly_maps_dw.append(day_hourly_map)

    # --- Vecchia용 데이터 로드 (day_offsets 인자 추가) ---
    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
        df_map, 
        day_offsets,  # <--- 이 부분이 추가되어야 합니다
        hour_indices, 
        ord_mm=ord_mm,
        dtype=torch.float64, 
        keep_ori= True
    )
    daily_aggregated_tensors_vecc.append(day_aggregated_tensor)
    daily_hourly_maps_vecc.append(day_hourly_map)

print(f"Aggregated Tensor Shape: {daily_aggregated_tensors_vecc[0].shape}")
# 예상 출력: torch.Size([행수, 12]) -> 열이 12개여야 성공입니다.

Aggregated Tensor Shape: torch.Size([22680, 12])


In [7]:
daily_aggregated_tensors_dw[0].shape

torch.Size([22680, 12])

In [9]:
v=0.5
mm_cond_number= 8
nheads = 300
patience, factor = 5, 0.5


In [11]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Global L-BFGS Settings
LBFGS_LR = 1.0
LBFGS_MAX_STEPS = 2      # 10 to 20  
LBFGS_HISTORY_SIZE = 100   
LBFGS_MAX_EVAL = 100       # line search from 50 to 80
       
DELTA_LAT, DELTA_LON = 0.044, 0.063 
LAT_COL, LON_COL, VAL_COL, TIME_COL = 0, 1, 2, 3

days_list = [4]

# --- 2. Run optimization loop over pre-loaded data ---

for day_idx in days_list:  # 0-based

    print(f'\n{"="*40}')
    print(f'--- Starting Processing for Day {day_idx+1} (2024-07-{day_idx+1}) ---')
    print(f'{"="*40}')

    # Assuming data access is correct
    daily_hourly_map_dw = daily_hourly_maps_dw[day_idx]
    daily_aggregated_tensor_dw = daily_aggregated_tensors_dw[day_idx]

    daily_hourly_map_vecc = daily_hourly_maps_vecc[day_idx]
    daily_aggregated_tensor_vecc = daily_aggregated_tensors_vecc[day_idx]

    if isinstance(daily_aggregated_tensor_vecc, torch.Tensor):
        daily_aggregated_tensor_vecc = daily_aggregated_tensor_vecc.to(DEVICE)


    init_sigmasq   = 13.059
    init_range_lat = 0.2
    init_range_lon = 0.25
    init_advec_lat = 0.0218
    init_range_time = 1.5
    init_advec_lon = -0.1689
    init_nugget    = 0.247

    #init_sigmasq   = 13.059
    #init_range_lat = 0.154 
    #init_range_lon = 0.195
    #init_advec_lat = 1e-8
    #init_range_time = 1e-8
    #init_advec_lon = 1e-8
    #init_nugget    = 0.247
    
    
    # Map model parameters to the 'phi' reparameterization
    init_phi2 = 1.0 / init_range_lon                # 1/range_lon
    init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
    init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
    init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

    # Create Initial Parameters (Float64, Requires Grad)
    initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                    np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

    params_list = [
        torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
        for val in initial_vals
    ]

    # --- 💥 Instantiate the GPU Batched Class ---
    # NOTE: Ensure fit_vecchia_lbfgs is the NEW class we defined
    model_instance = kernels_ar.fit_vecchia_lbfgs(
            smooth = v,
            input_map = daily_hourly_map_vecc,
            aggregated_data = daily_aggregated_tensor_vecc,
            nns_map = nns_map,
            mm_cond_number = mm_cond_number,
            nheads = nheads
        )

    '''  
    model_instance = kernels_reparam_space_time_gpu_col.fit_vecchia_lbfgs(
        smooth=v,
        #input_map=daily_hourly_maps_vecc_sim[0],
        #aggregated_data= daily_aggregated_tensors_vecc_sim[0],

        input_map=daily_hourly_maps_dw[2],
        aggregated_data= daily_aggregated_tensors_dw[2],

        nns_map=None,
        mm_cond_number=mm_cond_number,
        #nheads=nheads
    )
    '''

    # --- 💥 Set L-BFGS Optimizer ---
    optimizer_vecc = model_instance.set_optimizer(
                params_list,     
                lr=LBFGS_LR,            
                max_iter=LBFGS_MAX_EVAL,        
                history_size=LBFGS_HISTORY_SIZE 
            )

    print(f"\n--- Starting Phase 2: Vecchia Optimization (Day {day_idx+1}) ---")
    start_time = time.time()
    
    # --- 💥 Call the Batched Fit Method ---
    # REMOVED: model_instance.matern_cov_aniso_STABLE_log_reparam
    out, steps_ran = model_instance.fit_vecc_lbfgs(
            params_list,
            optimizer_vecc,
            # covariance_function argument is GONE
            max_steps=LBFGS_MAX_STEPS, 
            grad_tol=1e-7
        )


    end_time = time.time()
    epoch_time = end_time - start_time
    
    print(f"Vecchia Optimization finished in {epoch_time:.2f}s. Results: {out}")


Using device: cpu

--- Starting Processing for Day 5 (2024-07-5) ---

--- Starting Phase 2: Vecchia Optimization (Day 5) ---
🚀 Pre-computing (12 Cols Data with AR Offset Correction)... ✅ Done. (Heads: 2400, Tails: 20280)
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/2 / Loss: 1.075634 ---
  Param 0: Value=3.9444, Grad=2.5122850697929452e-06
  Param 1: Value=1.6562, Grad=-2.9718389451329234e-07
  Param 2: Value=-0.1963, Grad=9.73775054209972e-07
  Param 3: Value=-4.1816, Grad=-1.1505336841880973e-08
  Param 4: Value=0.0314, Grad=8.393514199372953e-07
  Param 5: Value=-0.0464, Grad=8.690571306757544e-07
  Param 6: Value=-0.7889, Grad=3.081961265149558e-08
  Max Abs Grad: 2.512285e-06
------------------------------
--- Step 2/2 / Loss: 1.054228 ---
  Param 0: Value=3.9444, Grad=2.5122850697929452e-06
  Param 1: Value=1.6562, Grad=-2.9718389451329234e-07
  Param 2: Value=-0.1963, Grad=9.73775054209972e-07
  Param 3: Value=-4.1816, Grad=-1.1505336841880973e-08
  Param 4: Val

In [ ]:
irr. (-3,2 121,131, intercept, time dummies, latitude)

Final Interpretable Params: {'sigma_sq': 14.557378105972255, 'range_lon': 0.36297285215444397, 'range_lat': 0.33986461719454436, 'range_time': 1.8307357609214716, 'advec_lat': -0.0014072484629318405, 'advec_lon': -0.16623085410420968, 'nugget': 0.24999425486908594}
Vecchia Optimization finished in 67.75s. Results: [3.691525186381127, 1.0134272349673297, 0.13156138084623087, -3.2362903523329813, -0.0014072484629318405, -0.16623085410420968, -1.3863173419076031, 0.9065025151721705]

regular (hide nugget)
------------------------------
------------------------------
Final Interpretable Params: {'sigma_sq': 13.881823879707401, 'range_lon': 0.3308191657110315, 'range_lat': 0.3048282266752421, 'range_time': 1.6460269337530522, 'advec_lat': -0.0014028377727820455, 'advec_lon': -0.15541841240605433, 'nugget': 1.112667285815519e-07}
Vecchia Optimization finished in 141.61s. Results: [3.7367637299362526, 1.1061833800531369, 0.16364694440410635, -3.209095690658186, -0.0014028377727820455, -0.15541841240605433, -16.01133555793201, 0.9490311091657573]